In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from time import clock

In [2]:
pages_to_scrape = 500
master_page = 'https://www.redkaraoke.com/karaoke/celine-dion/my-heart-will-go-on/11827/'
basePlayerUrl = "https://www.redkaraoke.com";
baseS3Url = "https://s3.amazonaws.com/";
#js file: https://d4rgvppl55fks.cloudfront.net/es/js/playerRecs.js

In [3]:
links_list = []
for i in range(pages_to_scrape):
    r = requests.get(master_page + 'recordings/p' + str(i))
    soup = BeautifulSoup(r.content, 'html.parser')
    for link in soup.find_all('a', attrs = {'class':'result result-recording'}):
        links_list.append('https://www.redkaraoke.com' + link.get('href'))
# Remove duplicated links
links_list_unique = set(links_list)

In [4]:
print(len(links_list))
print(len(links_list_unique))

12000
522


In [5]:
def extractUserRecording(soup):
# Localize data in "userRecording" object
    pattern = re.compile(r"userRecording")
    data = soup.find('script', text=pattern)

    pattern = re.compile(r"userRecording\.([a-zA-Z0-9]+)=([^;]+);")
    data = soup.find('script', text=pattern)
    ur_string = re.findall(r"userRecording\.([a-zA-Z0-9]+)=([^;]+);",str(data))
    ur_dict = dict(ur_string)
    for key in ur_dict.keys():
        ur_dict[key] = ur_dict[key][1:-1]
    return ur_dict 

In [6]:
def getDownloadLink(ur_dict):
    isDueto = False

    if 'isDueto' in ur_dict.keys(): 
        isDueto = True
    else:
        isDueto = False

    if ur_dict['file'][-1] == '3':
        if ur_dict['lang'] == 'es':
            if isDueto:
                filepath = basePlayerUrl + "/data/duets/mp3/es/"
            elif 's3' in ur_dict.keys():
                filepath = baseS3Url + "redkaraoke/data/mp3/"
            else:
                filepath = basePlayerUrl + "/data/mp3/es/"
        elif ur_dict['lang'] == 'jp':
            if isDueto:
                filepath = basePlayerUrl + "/data/duets/mp3/jp/"
            elif 's3' in ur_dict.keys():
                filepath = baseS3Url + "redkaraokejp/data/mp3/"
            else:
                filepath = baseS3Url + "/data/mp3/jp/"
        else:
            if isDueto:
                filepath = basePlayerUrl + "/data/duets/mp3/en/"
            elif 's3' in ur_dict.keys():
                filepath = baseS3Url + "redkaraokecom/data/mp3/"
            else:
                filepath = basePlayerUrl + "/data/mp3/en/"
    else:
        if ur_dict['lang'] == 'es':
            if isDueto:
                filepath = basePlayerUrl + "/data/duets/vid/es/"
            elif 'isDueto' in ur_dict.keys():
                filepath = baseS3Url + "redkaraoke/data/vid/"
            else:
                filepath = basePlayerUrl + "/data/vid/es/"
        elif ur_dict['lang'] == 'jp':
            if isDueto:
                filepath = basePlayerUrl + "/data/duets/vid/jp/"
            elif 's3' in ur_dict.keys():
                filepath = baseS3Url + "redkaraokejp/data/vid/"
            else:
                filepath = baseS3Url + "/data/vid/jp/"
        else:
            if isDueto:
                filepath = basePlayerUrl + "/data/duets/vid/en/"
            elif 's3' in ur_dict.keys():
                filepath = baseS3Url + "redkaraokecom/data/vid/"
            else:
                filepath = basePlayerUrl + "/data/vid/en/"
    download_link = filepath + ur_dict['file']
    return download_link

In [7]:
counter = 0

#Initialize lists
view_list = []
like_list = []
comment_list = []
recording_list = []
follower_list = []
following_list = []
date_list = []
gender_loc_list = []

location_list = []
user_avatar_list = []
user_group_list = []
# song_id_list = []
user_name_list = []
language_list = []
song_title_list = []
artist_list = []
download_link_list = []
recording_link_list = []

# Send request for each link in links_list_unique
for link_sub in links_list_unique:
    r = requests.get(link_sub)
    soup = BeautifulSoup(r.content, 'lxml')
    
    try:
        counter = counter + 1
        
        # Extract information from html code:
        view = int(soup.find('li', attrs = {'class':'rec-views'}).text)
        like = soup.find('li', attrs = {'class':'rec-likes'}).text
        comment = int(soup.find('li', attrs = {'class':'rec-comments'}).text)
        recording = int(soup.find('li', attrs = {'class':'tam1'}).find('strong').text)
        follower = int(soup.find_all('ul', attrs = {'class':'stats'})[0].find_all('li')[5].find('strong').text)
        following = (soup.find_all('ul', attrs = {'class':'stats'})[0].find_all('li')[6].find('strong').text)
        date = soup.find_all('div', attrs = {'class':'infoTema_textoBlanco'})[2].text[3:]
        gender_loc = soup.find('div', attrs = {'class':'userdata'}).find('p').text

        recording_link = link_sub

        ur_dict = extractUserRecording(soup)
        download_link = getDownloadLink(ur_dict)
        print(download_link)
        
        location = ur_dict['loc']
        user_avatar = ur_dict['uavatar']
        user_group = ur_dict['ugroup']
#         song_id = ur_dict['sid']
        user_name = ur_dict['u']
        language = ur_dict['lang']
        song_title = ur_dict['title']
        artist = ur_dict['a']
                
    except (IndexError, AttributeError) as e:
        continue
        
    view_list.append(view)
    like_list.append(like)
    comment_list.append(comment)
    recording_list.append(recording)
    follower_list.append(follower)
    following_list.append(following)
    date_list.append(date)
    gender_loc_list.append(gender_loc)
    
    location_list.append(location)
    user_avatar_list.append(user_avatar)
    user_group_list.append(user_group)
#     song_id_list.append(song_id)
    user_name_list.append(user_name)
    language_list.append(language)
    song_title_list.append(song_title)
    artist_list.append(artist)
    download_link_list.append(download_link)
    recording_link_list.append(recording_link)

    print(counter)

https://s3.amazonaws.com/redkaraokecom/data/vid/11827/937afc43a8ef8113a8b5c3165a87fe20_1.mp4
1
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/cc3a10d9e61120e35771413fc05b5209_1.mp4
2
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/32bae6d27f517cccb7b9c941c90d0f4b_1.mp4
3
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/0bb52160972f26033cb1ec3d22affb56_1.mp4
4
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/1f6543d928737c688c17b1ac7df4cab9_1.mp4
5
https://s3.amazonaws.com/redkaraoke/data/mp3/11827/4edd5715db8c579343d62893505953b0_1.mp3
6
https://s3.amazonaws.com/redkaraoke/data/mp3/11827/7d4fdb62f66acd461cabc0f27271e0cc_1.mp3
7
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/0e02a04a2c382b4b790dabda86e7c149_1.mp4
8
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/27985b3b2c0ee523e1b6f01dffb2dc82_1.mp4
9
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/54e8169e608f71e35e769a9092605cf6_1.mp4
10
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/8

https://s3.amazonaws.com/redkaraokecom/data/vid/11827/235035f4f9372a7d4847217792984cfb_1.mp4
95
https://www.redkaraoke.com/data/vid/es/11827/53c9308a0483ae2f64e8e8c7a0e74d85_1.mp4
96
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/8515962814aec9927f028d1f61f2013b_1.mp4
97
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/e6c59fb5bb2855542bdf33b41f7ddf92_1.mp3
98
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/429c3b5204386a594a9078f5195a4b4f_1.mp4
99
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/b6e68d8891e5d4558941215d38287c08_1.mp4
100
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/845b493c9831309195781fd5f5e749ab_1.mp4
101
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/a03c58f42ea5b5e1f70d3ef09efc1a7d_1.mp4
102
https://www.redkaraoke.com/data/vid/es/11827/b97a2e471ce857bc7402f6572fda8c2f_1.mp4
103
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/2dffe6185d8b9c5f8ea66d6bf476bb58_1.mp4
104
https://s3.amazonaws.com/redkaraokecom/data/vid/11827

https://s3.amazonaws.com/redkaraoke/data/mp3/11827/2e9d52218119b7df45d0207891be0344_1.mp3
184
https://s3.amazonaws.com/redkaraoke/data/mp3/11827/761ae335dfe28dab774787c9d5dda993_1.mp3
185
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/e4cd8a771cfae847df558707d78b8a8c_1.mp4
186
https://s3.amazonaws.com/redkaraokejp/data/vid/11827/1634ea2c7eb2ad2cbb9f0a1bce7986c3_1.mp4
188
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/519017fa346901095e8904a2fcb85eae_1.mp4
189
https://s3.amazonaws.com/redkaraokejp/data/vid/11827/ec78d9b0f7bb17ce53d42f181788b90d_1.mp4
190
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/f339b92ffc07dd1c520443543bc88044_1.mp4
191
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/408681e7e86ee992c0e38724c889cb55_1.mp4
192
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/65551c48ce75e158772c553eb2da288c_1.mp4
193
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/a88bf101b5cd615e2912317a7a6523bb_1.mp4
194
https://s3.amazonaws.com/redkaraokecom

https://s3.amazonaws.com/redkaraokecom/data/vid/11827/adecd4883cede5504263141aee122080_1.mp4
277
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/f8dcddb28c2f5b2a9cd82dcec2667360_1.mp4
278
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/de3c2c4141d067b9157e78a9c3e3cfa6_1.mp3
279
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/b4c1d48f77bbd28c7c4d1da8325728c8_1.mp4
280
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/d27d15b3523e4465d6843567b82aa01d_1.mp4
281
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/507f090c16c63519a3a3b22d71fd43df_1.mp4
282
https://s3.amazonaws.com/redkaraoke/data/mp3/11827/9bb223068ece9bdf4be04e1508936a0b_1.mp3
283
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/2f765b08a78e95a85da71a2140a65ec2_1.mp3
284
https://s3.amazonaws.com/redkaraoke/data/mp3/11827/5e998bf3eb925b12d70277c972fc030d_1.mp3
285
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/f3299341a24eedb82c026ea2acf9505a_1.mp4
286
https://s3.amazonaws.com/redkaraokec

https://s3.amazonaws.com/redkaraokecom/data/vid/11827/a8e9df9ebeff335e8f5d3d8da40fc610_1.mp4
367
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/c715da734ab9ad8193619b739e68fd44_1.mp3
368
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/a6346a9f8180e48b284c65fc5a81d761_1.mp4
369
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/00db445687555ce20a85642386d1bbbc_1.mp3
370
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/cd1ee4a53069c44e1d83baef7508b86b_1.mp4
371
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/6a3478d769f934b0b3d867601e5a65b0_1.mp4
372
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/b6f548ae6d6b0a5ad81e7e8f85fee1a3_1.mp4
373
https://s3.amazonaws.com/redkaraoke/data/mp3/11827/62cc0b2e8c15ed08b497709749e907f7_1.mp3
374
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/44430257a3462ee33b4f612265681fc9_1.mp4
375
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/6610eae97d76123031b32eabd587105b_1.mp3
376
https://s3.amazonaws.com/redkarao

https://s3.amazonaws.com/redkaraokecom/data/vid/11827/6d2b50c37093be446e04b56273da382a_1.mp4
458
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/b40d0993d8a2f344276c58ba40230200_1.mp3
459
https://www.redkaraoke.com/data/mp3/es/11827/639394b9606d470e23632a5cdbe65144_1.mp3
460
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/83ab547ed314be259a660f381d6aefc6_1.mp4
461
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/2d6fb2409ba38ea713987c0a84f35ed0_1.mp4
462
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/9dc969c4688ce6fc738bb6394c506383_1.mp4
463
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/a91edc1d7db779df2450fe4f83cd8d4b_1.mp4
464
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/ff5d0db835b5c185692fc3b92c27c84a_1.mp4
465
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/e616062f268df66760bf7cd6414c4870_1.mp3
466
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/6845ddb4439c668aae2957e041ad2b00_1.mp4
467
https://s3.amazonaws.com/redkaraokecom/

In [8]:
print(len(view_list))

490


In [9]:
d = {'no_of_views': view_list, 'no_of_likes': like_list, 'no_of_comments': comment_list, 
     'no_of_recordings': recording_list, 'no_of_followers': follower_list, 'no_of_following': following_list,
     'date_recorded': date_list, 'gender + location': gender_loc_list, 
     'location': location_list, 'user_avatar': user_avatar_list, 'user_group': user_group_list,
     'user_name': user_name_list, 'language': language_list, 'song_title': song_title_list, 
     'artist': artist_list, 'download_link': download_link_list, 'recording_link': recording_link_list}

In [10]:
df = pd.DataFrame(data=d)

In [11]:
df

,artist,date_recorded,download_link,gender + location,language,location,no_of_comments,no_of_followers,no_of_following,no_of_likes,no_of_recordings,no_of_views,recording_link,song_title,user_avatar,user_group,user_name
0,Celine Dion,03/26/2016,https://s3.amazonaws.com/redkaraokecom/data/vi...,USA,en,en,10,18,0,14,8,375,https://www.redkaraoke.com/profile/yonce123/re...,My heart will go on,https://d4rgvppl55fks.cloudfront.net/es/img3/n...,2,yonce123
1,Celine Dion,06/01/2010,https://s3.amazonaws.com/redkaraokecom/data/vi...,"Woman, Brooklyn, USA",en,en,40,49,0,61,24,1671,https://www.redkaraoke.com/profile/Gracellen/r...,My heart will go on,https://d4rgvppl55fks.cloudfront.net/forum/cus...,2,Gracellen
2,Celine Dion,01/25/2016,https://s3.amazonaws.com/redkaraokecom/data/vi...,"Man, Bern , Switzerland",en,en,4,45,38,7,14,190,https://www.redkaraoke.com/profile/kev90/recor...,My heart will go on,https://d4rgvppl55fks.cloudfront.net/forum/cus...,2,kev90
3,Celine Dion,12/19/2016,https://s3.amazonaws.com/redkaraokecom/data/vi...,Bangladesh,en,en,0,7,0,4,1,150,https://www.redkaraoke.com/profile/rdxsaykat9/...,My heart will go on,https://d4rgvppl55fks.cloudfront.net/es/img3/n...,2,rdxsaykat9
4,Celine Dion,01/30/2017,https://s3.amazonaws.com/redkaraokecom/data/vi...,Romania,en,en,7,17,15,6,47,194,https://www.redkaraoke.com/profile/davidaresan...,My heart will go on,https://d4rgvppl55fks.cloudfront.net/forum/cus...,2,davidaresan
5,Celine Dion,05/04/2016,https://s3.amazonaws.com/redkaraoke/data/mp3/1...,Spain,es,en,37,906,11,19,1829,118,https://www.redkaraoke.com/profile/Odisea3000/...,My heart will go on,https://d4rgvppl55fks.cloudfront.net/forum/cus...,9,Odisea3000
6,Celine Dion,06/10/2016,https://s3.amazonaws.com/redkaraoke/data/mp3/1...,"Woman, santo domingo, Dominican Republic",es,en,17,512,357,13,160,89,https://www.redkaraoke.com/profile/LINA__/reco...,My heart will go on,https://d4rgvppl55fks.cloudfront.net/forum/cus...,2,LINA__
7,Celine Dion,06/06/2017,https://s3.amazonaws.com/redkaraokecom/data/vi...,"Woman, JAKARTA , Indonesia",en,en,8,9,7,10,4,170,https://www.redkaraoke.com/profile/rehulinawat...,My heart will go on,https://d4rgvppl55fks.cloudfront.net/forum/cus...,2,rehulinawatibangun
8,Celine Dion,06/14/2016,https://s3.amazonaws.com/redkaraokecom/data/vi...,"Woman, occidental mindoro, Philippines",en,en,10,12,0,8,12,235,https://www.redkaraoke.com/profile/cute-angie/...,My heart will go on,https://d4rgvppl55fks.cloudfront.net/es/img3/m...,2,cute-angie
9,Celine Dion,01/14/2017,https://s3.amazonaws.com/redkaraokecom/data/vi...,"Woman, India",en,en,25,217,192,13,26,126,https://www.redkaraoke.com/profile/Kritikatalg...,My heart will go on,https://d4rgvppl55fks.cloudfront.net/forum/cus...,2,Kritikatalgotra


In [12]:
df.to_csv('./data/out.csv',index=False)